In [1]:
import ads

ads.set_auth(auth="resource_principal")

In [2]:
import tensorflow as tf

# Create the TensorFlow Model
mnist = tf.keras.datasets.mnist
(trainx, trainy), (testx, testy) = mnist.load_data()
trainx, testx = trainx / 255.0, testx / 255.0

model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10),
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(trainx, trainy, epochs=1)

11501568/11490434 [==============================] - 0s 0us/step


2025-11-25 19:14:40.352133: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-25 19:14:40.354409: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


1875/1875 [==============================] - 8s 4ms/step - loss: 0.2959 - accuracy: 0.9142


In [3]:
# Exercise/validate the prediction
model.predict(testx[0:1])

array([[ -0.90218353,  -7.333997  ,   0.79742765,   1.7919022 ,
         -7.7595043 ,  -1.8028702 , -10.595257  ,   8.217905  ,
         -2.7859497 ,   0.6825564 ]], dtype=float32)

In [4]:
## TENSORFLOW FRAMEWORK SERIALIZATION
# Create an ADS model
from ads.common.model_metadata import UseCaseType
from ads.model.framework.tensorflow_model import TensorFlowModel
from uuid import uuid4

tensorflow_model = TensorFlowModel(estimator=model, artifact_dir=f"./model-artifact-{str(uuid4())}")

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Available     Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Not Available Local tested .predict from score.py                               
save()    Not Available Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  Not Available Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [5]:
# Prepare the ADS Model
tensorflow_model.prepare(
    X_sample=trainx,
    y_sample=trainy,
    use_case_type=UseCaseType.MULTINOMIAL_CLASSIFICATION,
)

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Available     Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  Not Available Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [6]:
# Verify the ADS model
tensorflow_model.verify(testx)

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Start loading model.h5 from model directory /home/datascience/demo/model-artifact-cfa38f57-4fa4-4b61-a3a2-fbe90104022d ...
Model is successfully loaded.


Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done          Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  Not Available Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [7]:
# Save the ADS model
model_id = tensorflow_model.save()

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Start loading model.h5 from model directory /home/datascience/demo/model-artifact-cfa38f57-4fa4-4b61-a3a2-fbe90104022d ...
Model is successfully loaded.
['output_schema.json', 'score.py', 'model.h5', 'runtime.yaml']


loop1:   0%|          | 0/5 [00:00<?, ?it/s]

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done          Local tested .predict from score.py                               
save()    Done          Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  Available     Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [8]:
# Deploy and create an endpoint for the TensorFlow model
tensorflow_model.deploy(
    display_name="DIY TensorFlow Model For MNIST Classification",
    #deployment_log_group_id="ocid1.loggroup.oc1.xxx.xxxxx",
    #deployment_access_log_id="ocid1.log.oc1.xxx.xxxxx",
    #deployment_predict_log_id="ocid1.log.oc1.xxx.xxxxx",
    # Shape config details mandatory for flexible shapes:
    deployment_instance_shape="VM.Standard.E4.Flex",
    deployment_ocpus=1,
    deployment_memory_in_gbs=16,
)
print(f"Endpoint: {tensorflow_model.model_deployment.url}")

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

loop1:   0%|          | 0/6 [00:00<?, ?it/s]

Endpoint: https://modeldeployment.us-chicago-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.us-chicago-1.amaaaaaakoygqqiafmv2ili66ktwejcxyy7pto6sobd4jozsxjpfl2c4hgbq


Actions Needed
Step      Status    Details                                                           
initiate  Done      Initiated the model                                               
prepare() Done      Generated runtime.yaml                                            
                    Generated score.py                                                
                    Serialized model                                                  
                    Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done      Local tested .predict from score.py                               
save()    Done      Conducted Introspect Test                                         
                    Uploaded artifact to model catalog                                
deploy()  ACTIVE    Deployed the model                                                
predict() Available Called deployment predict endpoint

In [11]:
# Generate prediction by invoking the deployed endpoint
tensorflow_model.predict(testx[:3])['prediction']

[[-0.9021829962730408,
  -7.333996295928955,
  0.7974281311035156,
  1.7919025421142578,
  -7.759504318237305,
  -1.8028699159622192,
  -10.595255851745605,
  8.217904090881348,
  -2.785950183868408,
  0.6825563907623291],
 [-1.9111477136611938,
  -2.441387414932251,
  8.607331275939941,
  1.8207011222839355,
  -15.101482391357422,
  -0.4538317322731018,
  -2.363830804824829,
  -10.187736511230469,
  -2.9552433490753174,
  -14.652275085449219],
 [-5.241220951080322,
  4.963305473327637,
  -0.6726468205451965,
  -1.2769675254821777,
  -3.35022234916687,
  -3.399660348892212,
  -2.4540374279022217,
  -0.37900975346565247,
  -1.0223026275634766,
  -3.2069802284240723]]

In [12]:
# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Actions Needed
Step      Status    Details                                                           
initiate  Done      Initiated the model                                               
prepare() Done      Generated runtime.yaml                                            
                    Generated score.py                                                
                    Serialized model                                                  
                    Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done      Local tested .predict from score.py                               
save()    Done      Conducted Introspect Test                                         
                    Uploaded artifact to model catalog                                
deploy()  ACTIVE    Deployed the model                                                
predict() Available Called deployment predict endpoint

In [13]:
# Prepare data sample for prediction and save it to file 'data-payload'
from io import BytesIO
import base64
import numpy as np

data = testx[:3]
np_bytes = BytesIO()
np.save(np_bytes, data, allow_pickle=True)
data = base64.b64encode(np_bytes.getvalue()).decode("utf-8")
with open('data-payload', 'w') as f:
    f.write('{"data": "' + data + '", "data_type": "numpy.ndarray"}')

In [ ]:
# Test the deployment via the OCI CLI with the following shell commands
# $ export uri=https://modeldeployment.{region}.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.xxx.xxxxx/predict
# $ oci raw-request \
#    --http-method POST \
#    --target-uri $uri \
#    --request-body file://data-payload
#
# For this latest deployment this looks like this
# $ export uri=https://modeldeployment.us-chicago-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.us-chicago-1.amaaaaaakoygqqiafmv2ili66ktwejcxyy7pto6sobd4jozsxjpfl2c4hgbq/predict
# $ oci raw-request \
#    --http-method POST \
#    --target-uri $uri \
#    --request-body file://data-payload